In [1]:
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.passes import FullAncillaAllocation, EnlargeWithAncilla, ApplyLayout
from qiskit.converters import circuit_to_dag, dag_to_circuit
from utils import file_to_coupling_map, directory_to_circuits
import random
import time
import os
import numpy as np

In [2]:
seed = 42
random.seed(seed)

cmap_name= "heavy_hex_127.txt"
cmap = file_to_coupling_map(f"coupling_maps/{cmap_name}")
qc_name = "qft"
heuristic = "decay"
curr_seed = 42
qc_list = directory_to_circuits(f"circuits/{qc_name}")


In [3]:
pass_trials_list = [np.int64(1), np.int64(215), np.int64(464), np.int64(1000)]
print(pass_trials_list)

[np.int64(1), np.int64(215), np.int64(464), np.int64(1000)]


In [4]:
for qc in qc_list:
    print(f"{qc.name}")

qft_10.qasm
qft_29.qasm
qft_49.qasm
qft_68.qasm
qft_88.qasm
qft_107.qasm
qft_127.qasm


In [5]:
qc = qc_list[6]

num_qubits = qc.num_qubits
print(f"    Running {qc.name} ")

# CSV file to store the results
csv_path = f"results/trials_{qc_name}_127.csv"
# Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
with open(csv_path, "w") as f:
    f.write("swap_count,depth,runtime,num_qubits,heuristic,cmap,trials,seed\n")

    Running qft_127.qasm 


In [6]:
# Run the circuits for different number of trials
for pass_trials in pass_trials_list:
    print(f"Running {pass_trials} trials")
    # Create the sabre pass
    sabre_pass = SabreLayout(cmap, seed=curr_seed, max_iterations=4, swap_trials=pass_trials, layout_trials=pass_trials )
    dag = circuit_to_dag(qc)
    
    # Execute the pass manager
    start = time.time()
    dag_tr = sabre_pass.run(dag)
    end = time.time()
    # Collect the metrics for this trial
    runtime = end - start

    qc_tr = dag_to_circuit(dag_tr)
    swap_count = qc_tr.count_ops().get("swap", 0)
    depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2)

    # Write the results to the CSV file
    # Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
    with open(csv_path, "a") as f:
        f.write(f"{swap_count},{depth},{runtime},{num_qubits},{heuristic},{cmap_name},{pass_trials},{curr_seed}\n")
    print(f"            Swap count: {swap_count}, Depth: {depth}, Runtime: {runtime}")

Running 1 trials
            Swap count: 13625, Depth: 3861, Runtime: 28.892089128494263
Running 215 trials
